In [1]:
import boto3
from sagemaker.model import Model
from datetime import datetime

In [2]:
model_name = 'object-detection-2020-05-05-14-38'
sm = boto3.client('sagemaker')

resp = sm.describe_model(ModelName=model_name)
resp

{'ModelName': 'object-detection-2020-05-05-14-38',
 'PrimaryContainer': {'Image': '433757028032.dkr.ecr.us-west-2.amazonaws.com/object-detection:latest',
  'Mode': 'SingleModel',
  'ModelDataUrl': 's3://beyoung-sagemaker/coco-object-detection-20200427/output/object-detection-2020-04-27-02-50-13-765/output/model.tar.gz'},
 'ExecutionRoleArn': 'arn:aws:iam::476271697919:role/service-role/AmazonSageMaker-ExecutionRole-20200427T094649',
 'CreationTime': datetime.datetime(2020, 5, 21, 14, 39, 43, 511000, tzinfo=tzlocal()),
 'ModelArn': 'arn:aws:sagemaker:us-west-2:476271697919:model/object-detection-2020-05-05-14-38',
 'EnableNetworkIsolation': False,
 'ResponseMetadata': {'RequestId': '1517553d-b73a-49fc-af13-c7ac44784221',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1517553d-b73a-49fc-af13-c7ac44784221',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '577',
   'date': 'Thu, 21 May 2020 15:40:15 GMT'},
  'RetryAttempts': 0}}

In [3]:
sm_model = Model(model_data = resp['PrimaryContainer']['ModelDataUrl'], image = resp['PrimaryContainer']['Image'], 
                 role = resp['ExecutionRoleArn'])

In [ ]:
endpoint_name = datetime.now().strftime("object-detection-%Y-%m-%m-%H-%M")
sm_model.deploy(initial_instance_count=1, instance_type='ml.t2.medium', endpoint_name=endpoint_name, wait=False)
endpoint_name

# SageMaker Predictor

In [ ]:
from sagemaker.predictor import RealTimePredictor

In [ ]:
object_detector = RealTimePredictor(endpoint=endpoint_name)

In [ ]:
file_name = 'dog.jpg'
!wget -O {file_name} https://live.staticflickr.com/4106/5052728523_010830b44e_k_d.jpg

In [ ]:
with open(file_name, 'rb') as image:
    f = image.read()
    b = bytearray(f)

In [ ]:
import json

object_detector.content_type = 'image/jpeg'
results = object_detector.predict(b)
detections = json.loads(results)
print (detections)

In [ ]:
def visualize_detection(img_file, dets, classes=[], thresh=0.6):
        """
        visualize detections in one image
        Parameters:
        ----------
        img : numpy.array
            image, in bgr format
        dets : numpy.array
            ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
            each row is one object
        classes : tuple or list of str
            class names
        thresh : float
            score threshold
        """
        import random
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg

        img=mpimg.imread(img_file)
        plt.imshow(img)
        height = img.shape[0]
        width = img.shape[1]
        colors = dict()
        for det in dets:
            (klass, score, x0, y0, x1, y1) = det
            if score < thresh:
                continue
            cls_id = int(klass)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0 * width)
            ymin = int(y0 * height)
            xmax = int(x1 * width)
            ymax = int(y1 * height)
            rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                                 ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id],
                                 linewidth=1)
            plt.gca().add_patch(rect)
            class_name = str(cls_id)
            if classes and len(classes) > cls_id:
                class_name = classes[cls_id]
            plt.gca().text(xmin, ymin - 2,
                            '{:s} {:.3f}'.format(class_name, score),
                            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                    fontsize=12, color='white')
        plt.savefig('result.jpg', dpi=500)
        plt.show()

In [ ]:
object_categories = ['person', 'bicycle', 'car',  'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 
                     'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
                     'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
                     'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat',
                     'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                     'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot',
                     'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable',
                     'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven',
                     'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
                     'toothbrush']


In [ ]:
# Setting a threshold 0.20 will only plot detection results that have a confidence score greater than 0.20.
threshold = 0.30
# Visualize the detections.
visualize_detection(file_name, detections['prediction'], object_categories, threshold)

# Delete endpoint

In [ ]:
sagemaker.Session().delete_endpoint(object_detector.endpoint)